In [289]:
#dash_figures.py

import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
from io import StringIO

def empty_figure():
    """
    Function returning an empty figure with all the same template
    """
    fig = px.strip()
    # Set the background    
    fig.update_layout(
    plot_bgcolor='white'
    )
    fig.update_xaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )
    fig.update_yaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )
    fig.add_annotation(text="No data to show", x=0.5, y=0.5, showarrow=False,
                        font=dict(color="Black", size=20))
    return fig

def blank_figure():
    """
        Just creating the template
    """
    nutrients = ["fat_100g", "saturated-fat_100g", "carbohydrates_100g", "fiber_100g", "proteins_100g", "salt_100g", "macronutrients_100g"]

    # To put Energy and the nutrients on the same graph but with differents scales
    figure = make_subplots(specs=[[{"secondary_y": True}]])

    figure_energy = px.violin(box=False) 
    figure_others = px.violin(box=False) 

    figure_others.update_traces(width = 1)
        
    figure_energy.update_traces(marker = dict(color = "red"), hovertemplate='<br>Product name: %{customdata}<br>energy_100g = %{y}')
    figure_others.update_traces(marker = dict(color = "green"), hovertemplate='<br>Product name: %{customdata}<br>%{x}: %{y}')
            
            
    figure_energy_selected = px.strip()
    figure_others_selected = px.strip()
    figure_energy_selected.update_traces(hovertemplate='<br>Product name: %{customdata}<br>energy_100g = %{y}')
    figure_others_selected.update_traces(hovertemplate='<br>Product name: %{customdata}<br>%{x}: %{y}')
            
    for fig in [figure_energy_selected, figure_others_selected]:   
        fig.update_traces(marker = dict(color = "blue"), 
                            marker_size=8, name="Selected", 
                            marker_line_color="black", marker_line_width=2)
                         
    for i in range(len(figure_energy.data)):
        figure.add_trace(figure_energy.data[i], secondary_y=False)
        figure.add_trace(figure_others.data[i], secondary_y=True)
        figure.add_trace(figure_energy_selected.data[i], secondary_y=False)
        figure.add_trace(figure_others_selected.data[i], secondary_y=True)

    # Update of figure layout
    figure.update_layout(
        yaxis_title="g/100g",
        title=dict(text='', font=dict(size=24, color="black"), x=0.5, xanchor='center'),
        font=dict(size=18, color="black"),
        plot_bgcolor='white',
    )

        # Set y-axes titles and background
    figure.update_yaxes(
        title_text="g/100g (energy)", 
        secondary_y=False,
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey')

    figure.update_yaxes(
        title_text="g/100g (nutrients)", 
        secondary_y=True) 

    # Set x-axe ticks
    figure.update_xaxes(
        ticktext=["energy_100g"] + nutrients, 
        tickvals=[i for i in range(len(nutrients) + 1)],
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey')

    return figure

def figure_radar_plot(df_slice, nutrients, nutrients_choice, df_selected_products):

    # We retrieve the mediane for the nutrients
    if len(nutrients_choice) == 0 :
        median_df = df_slice[nutrients].median()
    else:
        median_df = df_slice[nutrients_choice].median()
    
    # To put Energy and the nutrients on the same graph but with differents scales
    #figure = make_subplots(specs=[[{"secondary_y": True}]])

    # We repeat the first at the end to close the radarplot
    values = median_df.values.tolist() + [median_df.values[0]]
    columns = median_df.index.tolist() + [median_df.index[0]]
    
    #figure_energy = px.line_polar(r = values_energy, theta = columns, markers=True)
    figure_radar = px.line_polar(r = values, theta = columns, markers=True)

    # To fill the plot
    figure_radar.update_traces(fill='toself',
                               line_color = "green",
                               name=f"Median of {df_slice.shape[0]} products",
                               showlegend = True)
    
    # Update radial axis range
    #figure_radar.update_polars(radialaxis=dict(range=[0, 100]))
    
    nb_selected_product = 0
    
    # if product(s) have been selected
    if df_selected_products is not None:
        if not isinstance(df_selected_products, pd.DataFrame):
            # Assuming df_selected_products is a JSON-formatted string
            df_selected_products = pd.read_json(StringIO(df_selected_products), orient='split')

        if df_selected_products.shape[0] > 0 :
            nb_selected_product = df_selected_products.shape[0]

            # Differents blue colors
            blue_colors = [
                "#0077cc", "#00b8ff", 
                "#009688", "#35a79c", "#54b2a9", "#65c3ba", "#83d0c9",
                "#daf8e3", "#97ebdb", "#00c2c7", "#0086ad", "#005582",
                "#77aaff", "#99ccff", "#bbeeff", "#5588ff", "#3366ff",
            ]

            # We add a radarplot trace for each
            for i in range(df_selected_products.shape[0]):

                if len(nutrients_choice) == 0 :
                    mask = df_selected_products.iloc[i][nutrients]
                else:
                    mask = df_selected_products.iloc[i][nutrients_choice]

                # We get the values
                values = mask.values.tolist() + [mask.values[0]]
                columns = mask.index.tolist() + [mask.index[0]]

                # We create the new trace
                selected_radar_figure = px.line_polar(r = values, theta = columns, markers=True)

                selected_radar_figure.update_traces(fill='toself')
                selected_radar_figure.update_traces(fill='toself',
                                                    line_color = blue_colors[i],
                                                    showlegend = True)

                full_name = df_selected_products.iloc[i].product_name

                truncated_name = full_name[:20]
                selected_radar_figure.update_traces(name = truncated_name,
                                          legendgroup = full_name)#,
    #                                      hovertemplate = full_name)

                # We add it to the main figure_radar
                figure_radar.add_trace(selected_radar_figure.data[0])

    # To change the order and put the Median on top
    figure_radar.data = figure_radar.data[::-1]
    figure_radar.update_layout(
        margin=dict(l=20, r=20, t=20, b=20),
        font=dict(color="black"),
    )
    
    taille = 1 if nb_selected_product < 3 else nb_selected_product/4
    
    figure_radar.update_layout(legend=dict(orientation="h", yanchor="bottom", y=-0.1*taille, xanchor="left", x=0.2,
                                          font=dict(size=10),itemwidth=30))
    # We change the hovertemplate names
    figure_radar.update_traces(
        hovertemplate='<br>%{theta} = %{r}' # nutrient = value
        )
        # Set the background  
    figure_radar.update_polars(bgcolor='white')
    figure_radar.update_layout(
        font_size = 15,
        polar = dict(
          bgcolor = "rgb(223, 223, 223)",
          angularaxis = dict(
            linewidth = 3,
            showline=True,
            linecolor='black'
          ),
          radialaxis = dict(
            side = "counterclockwise",
            showline = True,
            linewidth = 2,
            gridcolor = "white",
            gridwidth = 2,
          )
        ),
    )
    
    return figure_radar
        
        
        
def create_figure_products(df, list_nutrients, selected_nutrients, selected_graphical_type, df_selected_products):
    """
        V2 of the function creating the different graphics used in the dashboard
        Transitioning from a boxplot to a violinplot
        Each figures will have the full page
        Less conflicts, better integration in the dashboard.
        
        df: dataframe containing the data to project
        list_nutrients: columns projected (nutrients) 
        selected_nutrients: selected columns from the list_nutrients
        selected_graphical_type: type of graphic that will be projected
        selected_products: selected products that will be highlighted
    """
    
    # To put Energy and the nutrients on the same graph but with differents scales
    figure = make_subplots(specs=[[{"secondary_y": True}]])
    
    selected_nutrients = [] if selected_nutrients == None else selected_nutrients

    if selected_graphical_type in ["Distribution", "Products"]:
        if selected_graphical_type == "Distribution": 
            figure_energy = px.violin(df, y="energy_100g", box=False) 
            figure_others = (px.violin(df, y=selected_nutrients, box=False) 
                             if len(selected_nutrients) > 0 
                             else px.violin(df, y=list_nutrients, box=False))

            figure_others.update_traces(width = 1)

        elif selected_graphical_type == "Products":
            figure_energy = px.strip(df, y="energy_100g") 
            figure_others = (px.strip(df, y=selected_nutrients) 
                                 if len(selected_nutrients) > 0 
                                 else px.strip(df, y=list_nutrients))  
            
        figure_energy['data'][0]['customdata'] = [name for name in df['product_name']]
        if len(selected_nutrients) > 0 :
            figure_others['data'][0]['customdata'] = [name for name in df['product_name']] * len(selected_nutrients)
        else:
            figure_others['data'][0]['customdata'] = [name for name in df['product_name']] * len(list_nutrients)
        
        figure_energy.update_traces(marker = dict(color = "red"), hovertemplate='<br>Product name: %{customdata}<br>energy_100g = %{y}')
        figure_others.update_traces(marker = dict(color = "green"), hovertemplate='<br>Product name: %{customdata}<br>%{x}: %{y}')
                
                         
        if df_selected_products is not None:
            if not isinstance(df_selected_products, pd.DataFrame):
                # Assuming df_selected_products is a JSON-formatted string
                df_selected_products = pd.read_json(StringIO(df_selected_products), orient='split')

            figure_energy_selected = px.strip(df_selected_products, y="energy_100g") 
            figure_others_selected = (px.strip(df_selected_products, y=selected_nutrients) 
                                 if len(selected_nutrients) > 0 
                                 else px.strip(df_selected_products, y=list_nutrients))
            # Putting informations
            figure_energy_selected['data'][0]['customdata'] = [name for name in df_selected_products['product_name']]
            if len(selected_nutrients) > 0:
                figure_others_selected['data'][0]['customdata'] = [name for name in df_selected_products['product_name']] * len(selected_nutrients)
            else:
                figure_others_selected['data'][0]['customdata'] = [name for name in df_selected_products['product_name']] * len(list_nutrients)
            figure_energy_selected.update_traces(hovertemplate='<br>Product name: %{customdata}<br>energy_100g = %{y}')
            figure_others_selected.update_traces(hovertemplate='<br>Product name: %{customdata}<br>%{x}: %{y}')

        else :
            figure_energy_selected = px.strip()
            figure_others_selected = px.strip()
            
        for fig in [figure_energy_selected, figure_others_selected]:   
            fig.update_traces(marker = dict(color = "blue"), 
                                marker_size=8, name="Selected", 
                                marker_line_color="black", marker_line_width=2)
                         
        for i in range(len(figure_energy.data)):
            figure.add_trace(figure_energy.data[i], secondary_y=False)
            figure.add_trace(figure_others.data[i], secondary_y=True)
            figure.add_trace(figure_energy_selected.data[i], secondary_y=False)
            figure.add_trace(figure_others_selected.data[i], secondary_y=True)

        # Update of figure layout
        figure.update_layout(
            yaxis_title="g/100g",
            title=dict(text=f'Distribution of macronutrients of selected products [{df.shape[0]}]',
                       font=dict(size=24, color="black"), x=0.5, xanchor='center'),
            font=dict(size=18, color="black"),
            plot_bgcolor='white',
        )

        # Set y-axes titles and background
        figure.update_yaxes(
            title_text="g/100g (energy)", 
            secondary_y=False,
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey')
                             
        figure.update_yaxes(
            title_text="g/100g (nutrients)", 
            secondary_y=True) 
                             
        # Set x-axe ticks
        figure.update_xaxes(
            ticktext=["energy_100g"] + list_nutrients, 
            tickvals=[i for i in range(len(list_nutrients) + 1)],
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey')
            #range = [0, 100], tickmode="sync")
        return figure
                
    elif selected_graphical_type == "Radarplot":
        return figure_radar_plot(df, list_nutrients, selected_nutrients, df_selected_products)
    
    # Default, but shouldn't occur
    else:
        return empty_figure()

def patch_graphic(patched_figure, df_whole, df_product, ch_list_graph, nutrients_list, type_modification):
    """
        Function that modify the patch figure
        Take as arguments, the dataframes to use
        The type of figure
        The type of modification (list):
        A: Change for primary data
        B: Change to the product data
        
        Return the patched_figure
    """
    
    #A, B: For the positions in the graph: 1 and 2 are for the whole data, 3 and 4 for the selected produced
    for row in type_modification:
        if row == "A":
            df = df_whole.copy()
            patched_figure['layout']['title']['text'] = f'Distribution of macronutrients of {df.shape[0]} products'
            A, B = 0, 1

        elif row == "B":
            df = df_product.copy()
            A, B = 2, 3
        
        else:
            continue
        product_name_list = [[value] for value in df["product_name"].values] 
        patched_figure['data'][A]['customdata'] = product_name_list
        patched_figure['data'][B]['customdata'] = product_name_list * len(nutrients_list)

        patched_figure['data'][A]['y'] = [value for value in df["energy_100g"].values]
        patched_figure['data'][B]['x'] = [nut for nut in nutrients_list for value in df[nut].values]
        patched_figure['data'][B]['y'] = [value for nut in nutrients_list for value in df[nut].values]

    return patched_figure


def figure_result_model(df):
    """
        Function showing the probabilities (result) of the classification model (pnns1 or pnns2)
    """
    
    df_sorted = df.sort_values("probabilities")
    # To extract the pnns_groups (either 1 or 2) and the probabilities
    categories = list(df_sorted[df.columns[0]])
    probabilities = np.round(df_sorted[df_sorted.columns[1]]*100)

    figure = px.bar(x = probabilities, y = categories, orientation='h')

    # Update figure layout
    figure.update_layout(
                yaxis_title=None, xaxis_title="Probabilities to be correct (%)",
                title=dict(text=f'Results of prediction model from the image:',
                           font=dict(size=24, color="black"), x=0.5, xanchor='center'),
                font=dict(size=18, color="black"),
                plot_bgcolor='lightgray',
                paper_bgcolor='#F0F0F0',
            )
    figure.update_yaxes(showline=True, linecolor='black')
    figure.update_xaxes(showline=True, linecolor='black', range = [0, 100])

    # Update the hovertemplate information
    figure.update_traces(hovertemplate='<br>Pnns_groups: %{y}<br>Probability: %{x}%')

    return figure

In [2]:
#dash_components.py

from dash import html, dcc, dash_table

 # Function to generate a RangeSlider
def generate_slider(title, id_dcc, max_value):
    return html.Div([title,
                     dcc.RangeSlider(0, max_value, 1, value=[0, max_value],
                                    marks={0: "0", max_value: str(max_value)},
                                    id=id_dcc,
                                    tooltip={"placement": "bottom", "always_visible": True})
                     ], style={'textAlign': 'center', 'color': 'black', 'fontSize': 15})


# Function to generate a DropDown
def generate_dropdown(value, options, placeholder, multi, id_dcc, clearable = True):
    return dcc.Dropdown(
                value=value,
                options=options,
                style={'fontFamily': 'monospace', 'align-items': 'center', 'justify-content': 'center', 
                       'textAlign': 'center', 'color': 'black', 'fontSize': 15, 'width': '100%', 'cursor': 'pointer'},
                placeholder=placeholder,
                multi=multi,
                id=id_dcc,
                clearable=clearable,
            )
def generate_input(placeholder, id_dcc, type_dcc = "text"):
    return dcc.Input(
                style={'fontFamily': 'monospace', 'align-items': 'center', 'justify-content': 'center', 
                       'textAlign': 'center', 'color': 'black', 'fontSize': 15, 'width': '100%'},
                placeholder=placeholder,
                type=type_dcc,
                id=id_dcc,
            )

# Function to generate a button
def generate_button(title_button, id_button, style_button):
    return html.Button(
        title_button, 
        id=id_button, 
        n_clicks=0, 
        style=style_button)

# Function to generate a Dash table
def generate_table(df, page_size, id_table):
    return dash_table.DataTable(
        data=df,
        columns=None,
        page_size = page_size,
        style_as_list_view=True,
        style_header={'fontWeight': 'bold', 'color': 'black'},
        style_table={'overflowY': 'auto'},
        row_selectable='single',
        selected_rows=[],
        style_cell={
            'textAlign': 'center',
            'height': 'auto',
            'whiteSpace': 'normal'
        },
        id = id_table),

# Function to generate a radio item
def generate_radio_items(options, default, id_radio, inline = True):
    return dcc.RadioItems(
        value=default,
        options=[
            {'label': label, 'value': label} 
            for label in options
        ],
        inline=inline,
        id=id_radio,
        style={'textAlign': 'center', 'color': 'black', 'fontSize': 15, 'width': '100%'}
    )


In [4]:
# data_handling.py

import pandas as pd
import copy
import os
import dash
from dash import html
import requests
import concurrent.futures
import queue

# Return the pd.DataFrame
def get_data():
    return data

# Return a list with the name of the countries in the dataset
def get_unique_countries():
    # Options setup for dropdown of countries
    c1 = [country.split(",") for country in data.countries_en.unique()]
    c2 = [count for country in c1 for count in country]
    unique_countries = sorted(list(set(c2)))

    return unique_countries

# Return a list of dicts with the number of products by country
def products_by_countries():

    flags = {"United States":"🇺🇸", "France":"🇫🇷", "Germany":"🇩🇪", "United Kingdom":"🇬🇧"}
    
    nb_products_countries = [
        {
            'label': f"{flags[country]} {country} [{return_df(country).shape[0]} products]",
            'value': country
        } 
        for country in get_unique_countries()
    ]

    return nb_products_countries

# Return a list of the pnns_groups 1 and sorted
def get_pnns_groups_1():
    return sorted(data.pnns_groups_1.unique())

# Return a list of the pnns_groups 2 and sorted
def get_pnns_groups_2():

    pnns_groups_2 = []
    for pnns1 in get_pnns_groups_1():
        for pnns2 in sorted(data.loc[data.pnns_groups_1 == pnns1, 'pnns_groups_2'].unique()):
            if pnns2 not in ['unknown', 'Alcoholic beverages']:
                pnns_groups_2.append(pnns2)

    return pnns_groups_2


def get_pnns_groups():

    pnns_groups = {}
    for pnns1 in get_pnns_groups_1():
        pnns_groups[pnns1] = sorted(data.loc[data.pnns_groups_1 == pnns1, "pnns_groups_2"].unique().tolist())

    return pnns_groups


def pnns_groups_options(country, pnns_groups_num, pnns1=None):
    if pnns_groups_num == "pnns_groups_1":
        pnns_groups = data[pnns_groups_num].unique()
    elif pnns_groups_num == "pnns_groups_2":
        pnns_groups = data.loc[data.pnns_groups_1 == pnns1, pnns_groups_num].unique()

    # Create a DataFrame with counts for each pnns group
    counts_df = data.query('countries_en.str.contains(@country)').groupby(pnns_groups_num).size().reset_index(name='count')

    # Merge the counts with the unique pnns groups
    merged_df = pd.DataFrame({pnns_groups_num: pnns_groups})
    merged_df = pd.merge(merged_df, counts_df, on=pnns_groups_num, how='left').fillna(0)
    
    merged_df.sort_values(by=pnns_groups_num, inplace=True)
    
    # Create the pnns_groups_options list
    pnns_groups_options = [
        {
            'label': f"{pnns} [{count} products]",
            'value': pnns
        }
        for pnns, count in zip(merged_df[pnns_groups_num], merged_df['count'])
    ]

    return pnns_groups_options  

def cache(fun):
    cache.cache_ = {}

    def inner(country, pnns1, pnns2):
        # Check if the inputs have changed
        inputs_changed = (
            country not in cache.cache_ or
            pnns1 not in cache.cache_ or
            pnns2 not in cache.cache_
        )
        
        cache_key = (country, pnns1, pnns2)

        if inputs_changed or cache_key not in cache.cache_:
            cache.cache_[cache_key] = fun(country, pnns1, pnns2)

        return cache.cache_[cache_key]

    return inner

@cache
def function(country, pnns1, pnns2):
    
    df = data.query('countries_en.str.contains(@country)')
    
    if pnns2:
        df = df.query('pnns_groups_2 == @pnns2')
    elif pnns1:
        df = df.query('pnns_groups_1 == @pnns1')

    return df

def return_df(country, pnns1 = None, pnns2 = None):

    if country:
        return function(country, pnns1, pnns2)

    # Returning no data to show
    else:
        return None

def get_image(code, number = 1):
    # Transform the code to produce the Open Food Facts image URL
    # number = the image we choose to retrieve
    if len(code) <= 8:
        url = f'https://images.openfoodfacts.org/images/products/{code}/{number}.jpg'
        return url
    elif len(code) > 8:
        code = "0"*(13 - len(code)) + code
        url = f'https://images.openfoodfacts.org/images/products/{code[:3]}/{code[3:6]}/{code[6:9]}/{code[9:]}/{number}.jpg'
        return url
    else:
        return None

def check_url_image(url):
    """
    Check if the link of the image is correct
    Return none if not
    """
    
    try:
        response = requests.head(url) # We use head instead of get, we only need to check the link
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
        return url
    except requests.exceptions.RequestException:
        return None

def check_image_urls_parallel(urls):
    """
    The check_image_urls_parallel function uses concurrent.futures.ThreadPoolExecutor 
    to perform the checks concurrently.
    """
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        #The map function is used to apply the check_url function to each URL in parallel.
        results = executor.map(check_url_image, urls) # return a generator
        
    #The check_url_image function puts the results appropriate queue (checked_queue or failed_queue)
    for result in results:
        if result is not None:
            checked_queue.put(result)
        else:
            failed_queue.put(result)   
        
# Store globally informations on the url images.
# If one user check one image, it will be beneficial to all
# failed_img contains the url that failed.
checked_images = set()
failed_img = set()

# We set a queue for allowing a multi user environment 
checked_queue = queue.Queue()
failed_queue = queue.Queue()

def testing_img(images):
    to_test = []

    for i, url in enumerate(images):
        if url in checked_images:
            if url in failed_img:
                images[i] = None
        else: 
            if url is not None and 'http' in str(url): 
                checked_images.add(url)
                to_test.append(url)

    check_image_urls_parallel(to_test)
    
    # The queue helps us handle a multi users environement
    while not checked_queue.empty():
        url = checked_queue.get()
        checked_images.add(url)

    while not failed_queue.empty():
        url = failed_queue.get()
        failed_img.add(url)

    for i, url in enumerate(images):
        if url in failed_img:
            images[i] = None
      
    return images
        
        
def get_code(url):
    # Extract the product code from the Open Food Facts image URL
    try:
        parts = url.split('/')
        
        # Short code format
        if parts[-3] == 'products':
            return parts[-2]
        
        # Long code format
        else:
            code = parts[-5] + parts[-4] + parts[-3] + parts[-2]
            return code.lstrip('0')  # Remove leading zeros
    except:
        pass

    return None

def mapping_nutriscore_IMG(df):
    """
        Function matching the nutriscore to the letter A to E
        Add the column nutriscore_score_letter to the dataframe
        We loop inside the dictionnary, and change the letter if the score is inferior.
        Until it isn't
    """
    nutriscore_img = {
        "E" : ["40", "nutriscore_E.png"], 
        "D" : ["18", "nutriscore_D.png"],
        "C" : ["10", "nutriscore_C.png"],
        "B" : ["2", "nutriscore_B.png"],
        "A" : ["-1", "nutriscore_A.png"],
    }
    if isinstance(df, pd.DataFrame):   
        df["nutriscore_score_letter"] = "nutriscore_E.png"
        for letter in nutriscore_img:
            df.loc[: ,"nutriscore_score_letter"] = df.apply(lambda x: nutriscore_img[letter][1] 
                                                if x["nutriscore_score"] <= int(nutriscore_img[letter][0])
                                               else x["nutriscore_score_letter"], axis = 1)
        return df

def df_sorting(diet, df = None):
    """
        Function sorting the dataframe
        ascending = True
        Descending = False
    """

    type_diet = {
        "Healthier foods": [
            {'column_id': 'nutriscore_score', 'direction': True},
        ],
        "Fiber rich foods": [
            {'column_id': 'fiber_100g', 'direction': False},
            {'column_id': 'nutriscore_score', 'direction': True}
        ],
        "Low sugar foods": [
            {'column_id': 'carbohydrates_100g', 'direction': True},
            {'column_id': 'nutriscore_score', 'direction': True}
        ],
        "Protein rich foods": [
            {'column_id': 'proteins_100g', 'direction': False},
            {'column_id': 'nutriscore_score', 'direction': True}
        ],
        "Energy rich foods": [
            {'column_id': 'energy_100g', 'direction': False},
            {'column_id': 'nutriscore_score', 'direction': True}
        ],
        "Low fat foods": [
            {'column_id': 'fat_100g', 'direction': True},
            {'column_id': 'nutriscore_score', 'direction': True}
        ],
        "Low salt foods": [
            {'column_id': 'salt_100g', 'direction': True},
            {'column_id': 'nutriscore_score', 'direction': True}
        ],
        "Low saturated fat foods": [
            {'column_id': 'saturated-fat_100g', 'direction': True},
            {'column_id': 'nutriscore_score', 'direction': True}
        ],
    }
    column_id = []
    direction = []
    
    df_copy = copy.deepcopy(df)
    
    if diet in type_diet:
        for sorting_param in type_diet[diet]:
            
            column_id.append(sorting_param['column_id'])
            direction.append(sorting_param['direction'])
            
        if isinstance(df, pd.DataFrame):  
            df_copy = df_copy.sort_values(by=column_id, ascending=direction)
    
    if isinstance(df_copy, pd.DataFrame):
        return df_copy
    else: 
        return column_id

    
# Function filling the list subtitles, images, styles_images, textes_images
def generate_texte_image(df, diets, n_best, subtitles, images, styles_images, textes_images):                 
    for i, diet in enumerate(diets):
        subtitles[i] = html.Strong(f"{diet}")
        
        # We sort, we take the n_best then we map the nutriscore label
        df_N_best = df_sorting(diet, df).head(n_best)

        for y, (_, IMG) in enumerate(df_N_best.iterrows()):
            index = y if i == 0 else (20 * i) + y
            code = IMG.iloc[0]
            product_name = IMG.iloc[1]
            
            # We retrieve the image url via the code
            images[index] = get_image(str(code))

            styles_images[index] = {'width': '150px', 'height': '150px'}

            textes_images[index] = (
                html.Div([
                    get_nutriscore_image(str(IMG.iloc[-1]))
                ] + [
                html.Div(product_name, style={'text-align': 'center', 'margin-top': '5px'}) 
                ])
            ) 
    # Checking each images
    #images = check_image_urls_parallel(images)
    images = testing_img(images)
    # Replace image 
    images = [dash.get_asset_url('no_image.jpg') if url is None else url for url in images]
    
    return subtitles, images, styles_images, textes_images   

# Return image of nutriscore
def get_nutriscore_image(img, style={'width': '100px', 'height': '50px', 'margin-left':'10px'}):
    return html.Img(
            src=dash.get_asset_url(img),
                alt="Product Nutriscore",
                style=style
            )
    
    
# Return Image of nutriscore then the text below with nutrition informations
def get_texte_product(row):
    return html.Div([
        get_nutriscore_image(str(row.iloc[-1]))
        ] + [
        html.Div([
            html.Strong(f"{row.index[i]}:"),
            f" {row.iloc[i]}"],
            style={'text-align': 'left', 'margin-top': '1px', 'margin-left':'10px'}
            )
        for i, col in enumerate(row.iloc[:-1])
    ], style={'display': 'flex', 'flex-direction': 'column', 'width': '100%'})
    
def find_key_by_value(my_dict, value):
    """
    Function searching which key contains the value
    Args: The dict to search in and the value to search
    Return None, if not in the dict
    
    """
    
    for key, val in my_dict.items():
        if value in val:
            return key
    # If the value is not found
    return None 

try:
    # Get the current directory of the notebook
    file_dir = os.getcwd()
    app_dir = os.path.dirname(file_dir)

    # Define the path to the file in the /files directory
    #file_path = os.path.join(app_dir, 'files', 'cleaned_data.csv')
    file_path='cleaned_data.csv'
    # Now you can use the file_path to access your file
    with open(file_path, 'r') as file:
        data = pd.read_csv(file_path, sep = "\t")

except:
    # Get the directory of the script
    script_dir = os.path.dirname(os.path.abspath(__file__))

    # Go up one level to the parent directory (assuming the script is in the 'app' directory)
    app_dir = os.path.dirname(script_dir)

    # Define the path to the file in the /files directory
    file_path = os.path.join(app_dir, 'files', 'cleaned_data.csv')

    # Now you can use the file_path to access your file
    with open(file_path, 'r') as file:
        data = pd.read_csv(file_path, sep = "\t")

# We do the mapping of nutriscore
data = mapping_nutriscore_IMG(data)

/var/folders/2f/rfnwlqbn4kn5mss8s54qbyhh0000gn/T/ipykernel_59234/1167503542.py:394: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep = "\t")


In [315]:
import dash
from dash import Dash, html, dcc, Output, Input, State, ctx, Patch
import pandas as pd
import requests
import math
from io import StringIO
import time
from collections import Counter 

# Importing the functions
#from app.dash_figures import create_figure_products, figure_result_model
#from app.dash_components import generate_slider, generate_dropdown, generate_table, generate_radio_items, generate_input
#from app.data_handling import pnns_groups_options, return_df, get_image, get_code, df_sorting
#from app.data_handling import get_data, products_by_countries,get_pnns_groups_1, get_pnns_groups_2, get_pnns_groups
#from app.data_handling import generate_texte_image, get_texte_product, testing_img

# Linked to the external CSS file 
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css', '/assets/styles.css']

app = Dash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions=True)

#server = app.server

app.title = 'Nutritious app'

versionning = "version: 0.6.6"

DEBUG = True

products_availability = "Referenced products: " + str(get_data().shape[0])

nutrients = ["fat_100g", "saturated-fat_100g", "carbohydrates_100g", "fiber_100g", "proteins_100g", "salt_100g", "macronutrients_100g"]

slider_trigger = ["slider_energy", "slider_fat", "slider_saturated", "slider_carbohydrates", "slider_fiber", "slider_proteins", "slider_salt", "slider_macronutrients"]

diets = ["Healthier foods", "Fiber rich foods", "Low sugar foods", "Protein rich foods", "Low fat foods", "Low salt foods", "Low saturated fat foods", "Energy rich foods"]

# Default setup
default_country, default_pnns1, default_diet = "France", "Fruits and vegetables", "Healthier foods" 
default_graphic_option, default_search_option = "Distribution", "Product name"

# Generate list of unique countries and number of products
unique_countries = products_by_countries()

# Generate pnns_groups 1 and 2, and sorted
pnns_groups_1 = get_pnns_groups_1()

pnns_groups_2 = get_pnns_groups_2()

pnns_groups = get_pnns_groups()
  
# to handle increased number of diet
TOTAL_IMAGES = len(diets) * 20

# To have a clearer code

style24 = {'font-size': '24px', 'color': 'black', 'width': '100%', 
            'textAlign': 'center', 'margin': '0px', 'border': 'none', 'background-color': 'gray',
            'display': 'flex', 'flex-direction': 'column'}

style16 = {'font-size': '16px', 'color': 'black', 'width': '100%', 
        'textAlign': 'left', 'margin': '0px', 'border': 'none', 'background-color': 'gray'}

style16_nd = {'font-size': '16px', 'color': 'black', 'width': '100%', 
        'textAlign': 'center', 'margin': '0px', 'border': '1px solid black', 'background-color': 'lightgray'}

style15 = {'align-items': 'center', 'justify-content': 'center', 'border': '1px solid black',
        'font-size': '15px', 'color': 'black', 'width': '100%', 
        'textAlign': 'center', 'margin': '0px', 'background-color': 'white'}

style10 = {'font-size': '10px', 'color': 'black', 'width': '100%', 'display':'none', 
        'textAlign': 'left', 'margin': '0px', 'border': 'none', 'background-color': 'gray'}

app.layout = html.Div([
    # Left side
    html.Div(id='left_panel', children=[    
        html.Div(id='left_panel_div1', children=[  
            # For closing or opening the left panel
            html.Button('←', id='arrow_button_panel', 
                style={'font-size': '48px', 'padding': '0', 'margin': '0', 'border': 'none'})
        ]),
    
        html.Div(id='left_panel_div2', children=[     
            html.Div(
                html.Img(src=dash.get_asset_url('pomme.jpeg'), 
                     style={'width': '300px', 'height': '300px'}),
                style={'textAlign': 'center'}),

            # Title
            html.Div(className='row', children="Nutritious app",
                     style={'textAlign': 'center', 'color': 'black', 'fontSize': 48}),

            # Horizontale line
            html.Hr(style={'border-top': '4px solid black'}),  

                # Dropdown for the countries // Dropdown
            html.Div([
                generate_dropdown(default_country, unique_countries, "Choose a country", False, 'dropdown_country', False)
            ], style={'margin': '0 auto', 'border': '1px solid black'}),

            # Searchbar products // Dropdown
            html.Div([
                generate_dropdown(None, [], "Search a product", False, 'search_bar')
            ], style={'margin': '0 auto', 'border': '1px solid black', 'direction': 'ltr'}),
                            # RadioItems of graphic option
            html.Div([
                generate_radio_items(['Product name', 'Product code'], 
                                     default_search_option, 'type_search_product')
            ], style={'margin': '0 auto', 'border': '1px solid black', 'direction': 'ltr'}),

            # pnns_groups_search with an image // Button
            html.Div([
                generate_button("Picture search (in beta) 📷", "picture_search_button", style15)
            ], style={'margin': '0 auto'}),
            
            # Advanced searchbar products // Button
            html.Div([
                generate_button("Advanced search 🔍", "advanced_search_button", style15)
            ], style={'margin': '0 auto'}),
            
            # History button
            html.Div([
                generate_button("Browsing history", "browsing_button", style15)
            ], style={'margin': '0 auto', 'margin-bottom': '20px'}),

            html.Div([
                dcc.Loading(id="loading_section_pnns", type="default", children = [
                    html.Div([
                        generate_button(pnns1["label"], pnns1["value"], style16),
                        generate_button(pnns2["label"], pnns2["value"], style10)
                    ], style={'display': 'flex', 'flex-direction': 'column', 'width': '100%'}) 

                    if y == 0 and str(pnns1["value"]) != str(pnns2["value"]) else 

                        generate_button(pnns1["label"], pnns1["value"], style16)

                    if str(pnns1["value"]) == str(pnns2["value"]) else

                        generate_button(pnns2["label"], pnns2["value"], style10)

                    for pnns1 in pnns_groups_options(default_country, "pnns_groups_1")
                    for y, pnns2 in enumerate(pnns_groups_options(default_country, "pnns_groups_2", pnns1["value"]))
                ]),
            ]),

             dcc.Dropdown(
                id='dropdown_number_product',
                options=[
                    {'label': 'Displaying 5 products', 'value': 5},
                    {'label': 'Displaying 10 products', 'value': 10},
                    {'label': 'Displaying 15 products', 'value': 15},
                    {'label': 'Displaying 20 products', 'value': 20},
                ],
                clearable=False,
                value=10,  # Set the default value
                style={'font-size': '20px', 'color': 'black', 'cursor': 'pointer',  
                       'textAlign': 'center', 'border': '1px solid black'}
            ),

            # Informations
            html.Div([
                html.Div(className='row', children="Ruben HALIFA"),

                html.Div(className='row', children=versionning),

                html.Div(className='row', children=products_availability),
            ], style={'textAlign': 'left', 'color': 'black', 'fontSize': 12}),
        ]),
        ], style={'background-color': '#F0F0F0', 'overflowY': 'scroll', 'height': '100vh', 'flex': '1', 'direction': 'rtl',
                 'border-right': '1px solid black', 'margin':'0px'}),

    # Contents on the right side
    html.Div([

            html.Div([
                html.Div([html.Strong("ADVANCED SEARCH")], 
                                     style=style24),
                
                # Dropdown for the pnns_groups_1
                html.Div([
                    generate_dropdown(None, pnns_groups_1, "Choose a PNNS group 1 (optional)", False, 'dropdown_pnns1')
                ], style={'width': '75%', 'margin': '0 auto', 'margin-bottom': '20px'}),

                # Dropdown for the pnns_groups_2
                html.Div([
                    generate_dropdown(None, [], "Choose a PNNS group 2 (optional)", False, 'dropdown_pnns2')
                ], style={'width': '75%', 'margin': '0 auto', 'margin-bottom': '20px'}),
                
                # Input to search product
                html.Div([
                    generate_input("Search a product (e.g. 'milk') (optional)", "input_search_adv")
                ], style={'width': '75%', 'margin': '0 auto', 'margin-bottom': '20px'}),

                # Dropdown for the diet
                html.Div([
                    generate_dropdown(None, diets, "Choose a nutritious plan (optional)", False, 'dropdown_diet')
                ], style={'width': '75%', 'margin': '0 auto', 'margin-bottom': '20px'}),

                # Sliders controling which products we show
                html.Div([
                    generate_slider("Energy kcal/100g", 'slider_energy', 3880),
                    generate_slider("Fat g/100g", 'slider_fat', 100),
                    generate_slider("Saturated_fat g/100g", 'slider_saturated', 100),
                    generate_slider("Carbohydrates g/100g", 'slider_carbohydrates', 100),
                    generate_slider("Fiber g/100g", 'slider_fiber', 100),
                    generate_slider("Proteins g/100g", 'slider_proteins', 100),
                    generate_slider("Salt g/100g", 'slider_salt', 100),
                    generate_slider("Macronutrients g/100g", 'slider_macronutrients', 100)

                    ], style={'float': 'center', 'width': '500px', 'margin':'0 auto', 'flex-direction': 'row', 'margin-bottom': '20px'}),

                    # Button to reset options from the advanced search
                html.Div([
                    generate_button(html.Strong("Reset"), "reset_sliders_button", {}),
                ], style={'float': 'center', 'margin':'0 auto', 'textAlign': 'center', 'color': 'black', 'fontSize': 15, 'margin-bottom': '20px'}),

                # Button to confirm the search 
                html.Div([
                    generate_button("Search", "search_confirmation_button", {'width': '200px'})
                ], style={'float': 'center', 'font-size': '12px', 'color': 'black', 'width': '200px', 'margin':'0 auto', 
                           'textAlign': 'center', 'border': '1px solid black', 'background-color': 'lightgray'}),

                # Horizontale line
                html.Hr(style={'border-top': '4px solid black'}),  
                
            ], id = 'advanced_search_div', style={'float': 'center', 'display': 'None', 'flex-direction': 'row', 
                                              'width': '100%', 'background-color': '#F0F0F0', 'margin-bottom': '20px'}),

        
        html.Div([
            # Set an invisible anchor
            html.A(id="top_dash"),
            # To display a selected product at the top

            html.Div(id='selected_product_style', style={'display':'None'}, children=[
                dcc.Loading(id="loading_section_img", type="default", children = [
                    html.Div(id='selected_product_title',  
                        style=style24),

                    # Searchbar for products with the same product name
                    html.Div([
                        generate_dropdown(None, [], "Search a product", False, 'multiple_product_dropdown')
                    ], style={'margin': '0 auto', 'border': '1px solid black'}),
                    
                    html.Div([
                        html.Img(id='selected_product_img', src=dash.get_asset_url('no_image.jpg'), 
                            alt="No image available", style = {'height':'450px', 'width':'450px'}),
                        html.Div(id='selected_product_texte')
                    ], style={'display': 'flex', 'flex-direction': 'row', 'width': '100%'}),

                    # To display up to 3 + 1 alternatives images
                    html.Div([
                        html.A( 
                            html.Img(id=f"selected_img_{i}", src=dash.get_asset_url('no_image.jpg'), n_clicks = 0, 
                                 alt="No image available", style={'height': '150px', 'width': '150px'}),
                            href='#top_dash')
                        for i in range(4)
                    ], style={'display': 'flex', 'flex-direction': 'row', 'width': '100%'}),
                ]),
            ]),

            # To display the list of products

            html.Div(id='images_title', 
                             style=style24),

            html.Div([
                html.Div([
                    html.Button(children=f"{diet}", id=f"{diet}_div", n_clicks=0,
                                style={'font-size': '16px', 'color': 'black', 'textAlign': 'left', 'margin': '0px',
                                       'border': '1px', 'background-color': 'lightgray', 'width': '100%', 'margin': 'auto'}),
                    html.Div([
                        html.Div([
                            dcc.Loading(id=f"loading_section_{diet}_img_{i}", type="default", children = [
                            # html.A for having the clickable action on and going back to top
                            html.A( 
                                html.Img(id=f"{diet}_img_{i}", src=dash.get_asset_url('no_image.jpg'), n_clicks = 0, 
                                     alt="No image available", style={'height': '200px', 'width': '200px'}),
                            href='#top_dash'),
                            html.Div(id=f"{diet}_div_{i}")
                                ]),
                        ], style={'display': 'flex', 'flex-direction': 'column', 'width': '100%'})
                        for i in range(20)
                    ], style={'display': 'flex', 'flex-direction': 'row', 'overflowX': 'scroll'})
                    # Horizontal line
                ], style={'display': 'flex', 'flex-direction': 'column', 'width': '100%'})
                for diet in diets
            ]),

            html.Div(id='graphic_gestion', style={'display': 'None'}, children=[
                # Horizontale line
                html.Hr(style={'border-top': '4px solid black'}),  
                # RadioItems of graphic option
                html.Div([
                    generate_radio_items(['Distribution', 'Products'],  #'Radarplot', 
                                         default_graphic_option, 'check_list_graph_img')
                ], style={'margin': 'auto'}),

                # Dropdown for the macronutrient choice
                html.Div([
                    generate_dropdown(None, nutrients, "Choose nutrients", True, 'dropdown_nutrients_img')
                ], style={'margin': '0 auto'}),

                # Figure of macronutriments
                html.Div([
                    dcc.Graph(figure = blank_figure(), id="graph_products_img", style={'height': '600px', 'width': '100%', 'float': 'center'}),
                ], style={'display': 'flex', 'flex-direction': 'row', 'width': '100%'}),
            ]),


        ], style={'display': 'block', 'flex-direction': 'row', 'width': '100%'},  id='images_gestion'),

        # To display the browser history
        html.Div([
            html.Div(
                generate_table(None, 20, 'browser_table'),  
            )
        ], style={'display': 'None'},  id='browser_history_div'),
        
        # To display the search by picture
        
        html.Div([
            # Maximum MB = 15
            dcc.Upload([
                    generate_button(html.Strong("Upload image [max 15MB] (.JPEG, .PNG, .JPG)"), "upload_img_button", style16_nd),
                ], max_size = 15 * 1024 * 1024, # Maximum file size to 15MB
                   accept=".jpeg, .png, .jpg",  # Accepted file types
                   style={'margin': '0 auto', 'float': 'center'}, id="upload_img_data"),
    
            # To show the uploaded image
            html.Div(id='uploaded_img', style={'margin': '0 auto', 'text-align': 'center'}),
            
            # To have the 2 buttons on the same row
            html.Div([
                
                html.Div([
                    generate_button(html.Strong("Search pnns_groups_1"), "search_pnns1_img", {'display': 'None'}),
                ], style={'margin': '0 auto', 'text-align': 'left', 'width':'50%'}),
                
                html.Div([
                    generate_button(html.Strong("Search pnns_groups_2"), "search_pnns2_img", {'display': 'None'}),
                ], style={'margin': '0 auto', 'text-align': 'right', 'width':'50%'}),
                
            ], style={'display': 'flex', 'flex-direction': 'row', 'width': '100%'}),
            
            html.Div([
                    generate_button(html.Strong("Clear image"), "clear_img_button", {'display': 'None'}),
                ], style={'margin': '0 auto', 'float': 'center'}),
            
            # Div for the results
            html.Div([
                html.Div(style = {'margin-top': '10px'}),
            
                dcc.Graph(
                    id='model_figure_result',
                    config={'displayModeBar': False}
                ),

            ], style={'display':'None'}, id="result_model_image"),
            
            
        ], style={'display':'None'},
        id='picture_search_div')
            
    ], style={'flex-direction': 'row', 'width': '100%', 'background-color': '#F0F0F0', 
              'overflowY': 'scroll', 'height': '100vh', 'flex': '2'}),
    
    dcc.Store(id='sliced_file', data=None),
    dcc.Store(id='personnalized_sorting', data=None),
    dcc.Store(id='pnns1_chosen', data=None),
    dcc.Store(id='pnns2_chosen', data=None),
    dcc.Store(id='search_on', data=False),
    dcc.Store(id='shown_img', data={f'{diet}_img_{i}': None for diet in diets for i in range(20)}),
    # To store the client navigation history
    dcc.Store(id='history', data=[]),
    dcc.Store(id='loading_history', data=False),
    dcc.Store(id='prevent_update', data=False),
    dcc.Store(id='search_bar_data', data=False),
    
    
],id='app-container', style={'justify-content': 'space-between', 'margin': '0', 'padding': '0'})

@app.callback(
    Output('sliced_file', 'data'),
    
    Input('dropdown_country','value'),
    Input('dropdown_pnns1','value'),
    Input('dropdown_pnns2','value'),
    
    Input('advanced_search_button', 'n_clicks'),

    *[Input(f'{slide}', 'value') for slide in slider_trigger],
    
    prevent_initial_call=True,
)

def data_slicing(country, pnns1_chosen, pnns2_chosen, _, 
                 slide_energy, slide_fat, slide_sat_fat, slide_carbs, 
                 slide_fiber, slide_prot, slide_salt, slide_macro):

    sliders = [slide_energy, slide_fat, slide_sat_fat, slide_carbs, slide_fiber, slide_prot, slide_salt, slide_macro]
    elapsed_time = time.time() if DEBUG else None
      
    # Returning no data to show
    if country is None:
        return None

    # It follow the same path for all
    df = return_df(country, pnns1_chosen, pnns2_chosen)
    
    if ctx.triggered_id in slider_trigger:
        for nutrient, slide in zip(["energy_100g"] + nutrients, sliders):
            df = df[(df[nutrient] >= slide[0]) & (df[nutrient] <= slide[1])]
    
    # Transform to json format
    df = df.to_json(orient='split')
    
    print("Data slicing", time.time() - elapsed_time) if DEBUG else None
    return df

    
@app.callback(
    Output('search_bar', 'options'),
    Output('search_bar_data', 'data'),
    
    Input('dropdown_country','value'),
    Input('type_search_product','value'),
    Input('search_bar', 'search_value'), 
    
    State('search_bar_data', 'data'),
)

# We prepare the search_bar, changing when the country choice is different
def search_bar_option_def(country, dropdown_search, search_bar, search_bar_data):
    
    elapsed_time = time.time() if DEBUG else None
    
    if ctx.triggered_id in ['type_search_product', 'dropdown_country'] or search_bar_data is False:
        
        # If we change pnns or country group, we change the bar_option
        df = return_df(country, None, None)
        
        # If we search by product name
        if dropdown_search == 'Product name':
            # Get a Series with unique product names and their counts using Counter
            unique_counts = Counter(df['product_name'])

            # Sort the unique product names
            sorted_names = sorted(unique_counts.keys())

            # Create the search_bar_option list
            search_bar_data = [
                {
                    'label': f"{name} [{count} products]",
                    'value': name
                }
                for name in sorted_names
                for count in [unique_counts[name]]
            ]

        # If we search by product code
        elif dropdown_search == 'Product code':
            # Each product has its own unique code

            # Create the search_bar_option list
            search_bar_data = [
                {
                    'label': code,
                    'value': code
                }
                for code in df['code'].unique()
            ]
            
        search_bar_option = []
    
    # If user has written more than 2 letters or number, we show the selection
    elif ctx.triggered_id == 'search_bar':
        if len(search_bar) > 2:
            # We search for the products
            search_bar_option = [
                option for option in search_bar_data
                if search_bar.lower() in str(option['value']).lower()
                ]
        else:
            search_bar_option = []
        
        search_bar_data = dash.no_update    

    print("search_bar_option_def", time.time() - elapsed_time) if DEBUG else None
    return search_bar_option, search_bar_data

@app.callback(
    Output('multiple_product_dropdown', 'value'),
    Output('multiple_product_dropdown', 'options'),
    Output('multiple_product_dropdown', 'style', allow_duplicate=True),
    
    Input('search_bar', 'value'),
    State('type_search_product', 'value'),
    
    prevent_initial_call=True,
)

# When selected product have multiple possibilities 
def multiple_product_dropdown(search_bar, dropdown_search):
    # initialize values
    style_multiple_dropdown = dash.no_update
    option_multiple_dropdown = dash.no_update
    value_multiple_dropdown = None
    
    if dropdown_search == "Product name":
        
        # We search for all the products with the same name
        df_product = get_data().query('product_name == @search_bar')
        
        if df_product.shape[0] > 1:
            # We display the dropdown
            style_multiple_dropdown = {'display':'block'}
            
            # We put the first product
            value_multiple_dropdown = df_product.iloc[0]['code']
            
            # Style for the nutriscore image
            style={'width': '100px', 'height': '25px', 'margin-left':'10px'}
            
            # We put the code as value, as they are unique to each product
            option_multiple_dropdown = [
            {
                'label': html.Div([f"Product code: {row['code']}", 
                                   get_nutriscore_image(row['nutriscore_score_letter'], style)]),
                'value': row['code']
            }
            for _, row in df_product.iterrows()]
        
        
    return value_multiple_dropdown, option_multiple_dropdown, style_multiple_dropdown 

@app.callback(
    *[Output(f'{pnns2}', 'style') for pnns2 in pnns_groups_2],
    *[Output(f'{pnns1}', 'children') for pnns1 in pnns_groups_1],
    *[Output(f'{pnns2}', 'children') for pnns2 in pnns_groups_2],
    Output('advanced_search_div', 'style', allow_duplicate=True),
    Output('images_gestion', 'style', allow_duplicate=True),
    Output('pnns1_chosen', 'data'),
    Output('pnns2_chosen', 'data'),
    Output('history', 'data', allow_duplicate=True),
    
    *[Input(f'{pnns1}', 'n_clicks') for pnns1 in pnns_groups_1],
    *[Input(f'{pnns2}', 'n_clicks') for pnns2 in pnns_groups_2],
    Input('dropdown_country','value'),
    Input('loading_history','data'),
    Input('pnns1_chosen', 'data'),
    Input('pnns2_chosen', 'data'),
    
    State('history', 'data'),
    
    prevent_initial_call=True,
)
def click_pnns_showing(*args):
    elapsed_time = time.time() if DEBUG else None
    
    pnns2_option_visible = {'font-size': '10px', 'color': 'black', 'width': '100%', 'display':'block', 
              'textAlign': 'left', 'margin': '0px', 'border': 'none', 'background-color':'#C5C5C5'}
    
    pnns2_option_invisible = {'display': 'none'}
    
    pnns2_clicked_on = {'font-size': '16px', 'color': 'black', 'width': '100%', 'display':'block', 
              'textAlign': 'left', 'margin': '0px', 'border': 'none', 'background-color':'#C5C5C5'}
    
    style_search_div = {'display': 'None'}
    
    # Display go to default for the right panel (set mainly for when we are in the browser history)
    style_images_gestion = {'display': 'block', 'flex-direction': 'row', 'width': '100%'}
    
    # We retrieve the last arguments
    country, history_nav, pnns2_chosen, pnns1_chosen = args[-5], args[-1], args[-2], args[-3] 
    
    # Initialize the display to none
    output_style = [pnns2_option_invisible] * len(pnns_groups_2) 
    output_label_pnns1 = [dash.no_update] * len(pnns_groups_1)
    output_label_pnns2 =  [dash.no_update] * len(pnns_groups_2)
    
    # Function changing the style of the pnns buttons
    def changing_style_pnns_button(output_style, pnns):
        i = 0
        for pnns1 in pnns_groups_1:
            for pnns2 in pnns_groups[pnns1]:
                if pnns1 == pnns2:
                    continue

                # if it is a pnns groups 1 that was clicked on
                elif pnns == pnns1:
                    output_style[i] = pnns2_option_visible

                # if it is a pnns groups 2 that was clicked on
                elif pnns in pnns_groups[pnns1]:
                    # we highlight the one clicked on
                    if pnns == pnns2:
                        output_style[i] = pnns2_clicked_on
                    else:
                        output_style[i] = pnns2_option_visible
                i += 1 # We increment
                
        return output_style
    
    # When a pnns_groups_1 or 2 was clicked on
    if ctx.triggered_id in pnns_groups_1 + pnns_groups_2:  
        output_style = changing_style_pnns_button(output_style, ctx.triggered_id)
        
    # To modify style when navigating with a selected product
    elif ctx.triggered_id in ["pnns1_chosen", "pnns2_chosen"]:
        try:
            output_style = changing_style_pnns_button(output_style, pnns2_chosen)
        except:
            output_style = changing_style_pnns_button(output_style, pnns1_chosen)
    
    # When a pnns_groups_1 was clicked on
    if ctx.triggered_id in pnns_groups_1:
        
        pnns1_chosen = ctx.triggered_id
        pnns2_chosen = None
        
        # We add to the navigation history
        history_nav.insert(0, ["Navigation", country, ctx.triggered_id, pnns2_chosen, None, None])
    
    # When a pnns_groups_2 was clicked on
    elif ctx.triggered_id in pnns_groups_2:
        
        # We add to the navigation history
        history_nav.insert(0, ["Navigation", country, pnns1_chosen, ctx.triggered_id, None, None])
        
        pnns1_chosen = dash.no_update
        pnns2_chosen = ctx.triggered_id
                    
    # If loading from the browser history
    elif ctx.triggered_id == 'loading_history':
        # We retrieve the data
        pnns1_chosen, pnns2_chosen = args[-2], args[-1]
        
        # We check the pnns2 before pnns1
        pnns = pnns2_chosen if pnns2_chosen != None else pnns1_chosen
        
        # Modify the style of buttons
        output_style = changing_style_pnns_button(output_style, pnns)      
    
    # The country was change, we are modifying the title
    if ctx.triggered_id == 'dropdown_country':
        
        pnns1_chosen, pnns2_chosen = None, None
        
        # We add to the navigation history
        history_nav.insert(0, ["Navigation", country, pnns1_chosen, pnns2_chosen, None, None])
        
        # Product calcul
        pnns_groups_1_options = pnns_groups_options(country, "pnns_groups_1")
        
        # New label (= new number of products)
        output_label_pnns1 = [pnns1["label"] for pnns1 in pnns_groups_1_options]
        
        output_label_pnns2 = [
            pnns2["label"]
            for pnns1 in pnns_groups_1_options
            for pnns2 in pnns_groups_options(country, "pnns_groups_2", pnns1["value"])
            if str(pnns1["value"]) != str(pnns2["value"])
        ]
                        
    print("click_pnns_showing", time.time() - elapsed_time) if DEBUG else None
    
    output_values = [*output_style, *output_label_pnns1, *output_label_pnns2]
    output_values.extend([style_search_div, style_images_gestion, pnns1_chosen, pnns2_chosen, history_nav])

    return tuple(output_values)

@app.callback(
    *[
    Output(f"{slide}", property)
    for slide in slider_trigger
    for property in ['min', 'max', 'marks', 'value']
    ],
    Output("dropdown_pnns2", "options"), 
    Output("dropdown_pnns1", "value"), 
    Output("dropdown_pnns2", "value"), 
    Output("dropdown_diet", "value"), 
    
    Input("dropdown_pnns1", "value"), 
    Input("dropdown_pnns2", "value"), 
    Input('reset_sliders_button', 'n_clicks'),
    Input('advanced_search_button', 'n_clicks'),
    State('dropdown_country','value'),
    prevent_initial_call=True,
)
def update_sliders(pnns1_chosen, pnns2_chosen, n_clicks, n_clicks_search, country):
    elapsed_time = time.time() if DEBUG else None
            
    if ctx.triggered_id == "reset_sliders_button":
        pnns1_chosen, pnns2_chosen = None, None 
        pnns_groups_2 = []
        diet = default_diet
        # If the data change, we change the sliders
        df = return_df(country, pnns1_chosen, pnns2_chosen)
    
    else:
        df = return_df(country, pnns1_chosen, pnns2_chosen)
        if pnns1_chosen:
            pnns_groups_2 = pnns_groups[pnns1_chosen]
        else: 
            pnns_groups_2 = []
        diet = dash.no_update
    
    if isinstance(df, pd.DataFrame):
        output_values = []

        # Rounding down
        for nutrient in ["energy_100g"] + nutrients:
            nutrient_min = math.floor(df[f'{nutrient}'].min())
            nutrient_max = math.ceil(df[f'{nutrient}'].max())
            nutrient_marks = {nutrient_min: str(nutrient_min), nutrient_max: str(nutrient_max)}
            output_values.extend([nutrient_min, nutrient_max, nutrient_marks, [math.floor(nutrient_min), math.ceil(nutrient_max)]])

        print("update_sliders", time.time() - elapsed_time) if DEBUG else None
        output_values.extend([pnns_groups_2, pnns1_chosen, pnns2_chosen, diet])
        
        return tuple(output_values)

    return dash.no_update
    

@app.callback(
    Output('images_title', 'children'),
    *[Output(f'{diet}_div', 'children') for diet in diets],
    *[Output(f'{diet}_img_{i}', 'src') for diet in diets for i in range(20)],
    *[Output(f'{diet}_img_{i}', 'style') for diet in diets for i in range(20)],
    *[Output(f'{diet}_div_{i}', 'children') for diet in diets for i in range(20)],
    *[Output(f"selected_img_{i}", 'src') for i in range(4)],
    *[Output(f"selected_img_{i}", 'style') for i in range(4)],
    Output("graph_products_img", 'figure', allow_duplicate=True),
    Output("graphic_gestion", 'style'),
    Output('personnalized_sorting', 'data', allow_duplicate=True),
    Output("selected_product_style", 'style'),
    Output("selected_product_img", 'src'),
    Output("selected_product_title", 'children'),
    Output("selected_product_texte", 'children'),
    Output('advanced_search_div', 'style', allow_duplicate=True),
    Output('search_on', 'data'),
    Output('shown_img', 'data'),
    Output('history', 'data', allow_duplicate=True),
    Output('multiple_product_dropdown', 'style', allow_duplicate=True),
    Output('browser_history_div', 'style', allow_duplicate=True),
    Output('prevent_update', 'data'),
    Output('pnns1_chosen', 'data', allow_duplicate=True),
    Output('pnns2_chosen', 'data', allow_duplicate=True),
    Output('picture_search_div', 'style', allow_duplicate=True),
    
    *[Input(f'{diet}_div', 'n_clicks') for diet in diets],
    *[Input(f'{diet}_img_{i}', 'n_clicks') for diet in diets for i in range(20)],
    Input('pnns1_chosen', 'data'),
    Input('pnns2_chosen', 'data'),
    Input('dropdown_country','value'),
    Input('search_bar', 'value'),
    Input('search_confirmation_button', 'n_clicks'),
    Input('advanced_search_button', 'n_clicks'),
    Input('multiple_product_dropdown', 'value'),
    
    State('type_search_product', 'value'),
    State('dropdown_nutrients_img', 'value'),
    State('check_list_graph_img','value'),
    State('personnalized_sorting', 'data'),
    State('sliced_file', 'data'),
    State("dropdown_diet", "value"), 
    State('search_on', 'data'),
    State('dropdown_number_product', 'value'),
    State('shown_img', 'data'),
    State('history', 'data'),
    State('prevent_update', 'data'),
    State('input_search_adv', 'value'),
    
    prevent_initial_call=True,
)

def display_images(*args):
    elapsed_time = time.time() if DEBUG else None
    
    # We unpack the args
    (pnns1_chosen, pnns2_chosen, country, search_bar, clicked_search, click_advanced_search, value_multiple_dropdown,
     type_search_product, nutrients_choice, ch_list_graph, selected_diet, df_slice, dropdown_diet, search_on, 
     n_best, shown_img_data, history_nav, prevent_update, user_input) = args[-19:]
    
    # Return true if one of the diet was click.
    # It is helping for the browser history
    
    clicked_diet_ctx = [f'{diet}_div' for diet in diets]
    browser_diet = any(any(keyword in item['prop_id'] for keyword in clicked_diet_ctx) for item in ctx.triggered)
    
    # Initialize 
    no_display = {'display':'None'}
    images, styles_images, textes_images = [dash.no_update] * TOTAL_IMAGES, [no_display] * TOTAL_IMAGES, [None] * TOTAL_IMAGES
    others_img, others_img_style = [None] * 4, [no_display] * 4
    subtitles, title, figure = [None] * len(diets), None, dash.no_update 
    selected_product_img, selected_product_title, selected_product_texte = None, None, None
    graphic_gestion_style, selected_product_style, advanced_search_style = no_display, no_display, no_display
    search_on = False if search_on else dash.no_update
    selected_diet = None if selected_diet != None else dash.no_update
    style_multiple_dropdown = no_display
    style_picture_search_div = no_display
    browser_history_style = no_display
    
     # We take a Patch() to modify only some elements of the figure
    patched_figure = Patch()
    nutrients_choice = nutrients_choice if nutrients_choice not in [None, []] else nutrients 
    
    # Setting some conditions
    condition_selected_a_product = ctx.triggered_id in ["search_bar", "multiple_product_dropdown"] + [f'{diet}_img_{i}' for diet in diets for i in range(20)]
    condition_selected_diet_search_navigation = (ctx.triggered_id in clicked_diet_ctx + ['search_confirmation_button']) or (browser_diet)
    
    # dataframe preparation, only when necessary
    if ctx.triggered_id not in ['advanced_search_button', 'search_confirmation_button']:
        df = return_df(country, pnns1_chosen, pnns2_chosen)
        
    elif ctx.triggered_id == 'search_confirmation_button':
        df = pd.read_json(StringIO(df_slice), orient='split', dtype={'code': str})
        # We adjust if the user has entered a value
        if user_input is not None:
            df = df.query('product_name.str.contains(@user_input)')
        
    # If clicking on diet preference or confirmed search or from the browser
    if condition_selected_diet_search_navigation: 
        
        for i, diet in enumerate([diet + "_div" for diet in diets]):
            subtitles[i] = html.Strong(f"{diets[i]}")
            
            if (ctx.triggered_id in ['search_confirmation_button', diet]) or browser_diet: 
                
                # If client clicked on the confirmation button of the advanced search
                if ctx.triggered_id == 'search_confirmation_button':
                    # For the selected diet
                    if diet[:-4] == dropdown_diet: #:-4 is to eliminate some text
                        selected_diet = diets[i] # To keep the selected button
                        advanced_search_style = dash.no_update
                        search_on = True
                        
                    # If none selected
                    elif (dropdown_diet == None) or (dropdown_diet == []):
                        selected_diet = "All"
                        advanced_search_style = dash.no_update
                        search_on = True
                
                # Elif client clicked on one of the diet button
                else:
                    # We check which one is in ctx.triggered
                    result = any(diet in item.get('prop_id', '') for item in ctx.triggered)
                    
                    # Add information when it is the selected diet
                    if (diet == ctx.triggered_id) or (result):
                        selected_diet = diets[i] # To keep the selected button
                        
                        # We add to the navigation history
                        history_nav.insert(0, ["Navigation", country, pnns1_chosen, pnns2_chosen, diets[i], None])
                
                if selected_diet == diets[i]:
                    
                    title = html.Strong(f"BEST RECOMMENDED PRODUCTS FOR {diets[i].upper()}")

                    # sort and retrieve the N best, then match the nutriscore image           
                    df_N_best = df_sorting(diets[i], df).head(n_best)
                    
                    for y, (_, row) in enumerate(df_N_best.iterrows()):
                        index = y if i == 0 else (20 * i) + y

                        styles_images[index] = {'height': '400px', 'width': '400px'}
                        
                        # generate the texte below the image
                        textes_images[index] = get_texte_product(row)
                        
                        # We retrieve the image url via the code
                        images[index] = get_image(str(row.iloc[0]))
                        
                    # Checking each images
                    images = testing_img(images)
                    # Replace images
                    images = [dash.get_asset_url('no_image.jpg') if url is None else url for url in images]
                            # Creating a figure of the data distribution 
                    
                    figure = patch_graphic(patched_figure, df, df_N_best, 
                                                           ch_list_graph, nutrients_choice, ["A", "B"])

                    graphic_gestion_style = {'display':'block'}
        
                # When the user doesn't specifie a diet to search on
                elif selected_diet == "All":
                    subtitles, images, styles_images, textes_images = generate_texte_image(df, diets, n_best, 
                                                                               subtitles, images, 
                                                                               styles_images, textes_images)
                    # Checking each images
                    images = testing_img(images)
                    # Replace images
                    images = [dash.get_asset_url('no_image.jpg') if url is None else url for url in images]

        prevent_update = None
        
    # When navigating on the left panel    
    elif ctx.triggered_id in ['pnns1_chosen', 'pnns2_chosen', 'dropdown_country']:
        
        title = html.Strong("BEST RECOMMENDED PRODUCTS BY CATEGORY")
        
        subtitles, images, styles_images, textes_images = generate_texte_image(df, diets, n_best, 
                                                                               subtitles, images, 
                                                                               styles_images, textes_images)
        
        # Checking each images
        images = testing_img(images)
        # Replace images
        images = [dash.get_asset_url('no_image.jpg') if url is None else url for url in images]
        
        prevent_update = pnns2_chosen if ctx.triggered_id == "pnns2_chosen" else None
                                
    # If the client clicked on the search bar or one of the picture
    elif condition_selected_a_product:
        
        try:
            # If searched by the search bar
            if ctx.triggered_id == "search_bar":
                # We check if it is a product name or code entered

                if type_search_product == "Product name":
                    product_code = get_data().query('product_name == @search_bar').get('code')
                    # If more than 1 product
                    if product_code.shape[0] > 1:
                        code = str(value_multiple_dropdown)
                        style_multiple_dropdown = {'display':'block'}
                    else:
                        code = product_code.values[0]

                elif type_search_product == "Product code":
                    code = str(search_bar)

            # If searched by the intra search bar (multiple products with the same name)
            elif ctx.triggered_id == "multiple_product_dropdown":
                # We get the code
                code = str(value_multiple_dropdown)
                style_multiple_dropdown = {'display':'block'}

            # If searched by clicking on image
            else: 
                url = shown_img_data[ctx.triggered_id]
                code = get_code(url)

            df_product = get_data().query('code == @code')

            # For now it helps to deal with the 0 problem.
            if df_product.shape[0] == 0:
                code = "0"*(13 - len(code)) + code
                df_product = get_data().query('code == @code')

            # We get the product's name
            product_name = df_product['product_name'].values[0]

            # Principale image
            selected_product_img = get_image(code)

            # Secondaries images
            # Return the link of the image, then check it's validity
            
            others_img = [get_image(code, i) for i in range(1, 5)]
            others_img = testing_img(others_img)
            

            # Display image if link correct
            others_img_style = [{'height': '150px', 'width': '150px'}
                               if others_img[i] != None else no_display
                               for i in range(0, 4)]

            selected_product_title = html.Strong(product_name)
            selected_product_texte = get_texte_product(df_product.iloc[0])

            pnns1 = df_product["pnns_groups_1"].values[0]
            pnns2 = df_product["pnns_groups_2"].values[0]

            # We add to the navigation history
            history_nav.insert(0, [f"Product: {product_name}", country, pnns1, pnns2, None, code])

            # If the product visualize is the same as before, we prevent some of the front end update
            if pnns2 == prevent_update:

                title = dash.no_update
                subtitles = [dash.no_update] * len(diets)
                styles_images = [dash.no_update] * TOTAL_IMAGES
                textes_images = [dash.no_update] * TOTAL_IMAGES

                figure = patch_graphic(patched_figure, None, df_product, 
                                               ch_list_graph, nutrients_choice, ["B"])

            else:
                df = return_df(country, pnns1, pnns2).copy()

                title = html.Strong("BEST RECOMMENDED PRODUCTS BY CATEGORY")

                subtitles, images, styles_images, textes_images = generate_texte_image(df, diets, n_best, 
                                                                                       subtitles, images, 
                                                                                       styles_images, textes_images)
                # Checking each images
                images = testing_img(images)
                # Replace images
                images = [dash.get_asset_url('no_image.jpg') if url is None else url for url in images]
                
                # Creating a figure of the data distribution 
                figure = patch_graphic(patched_figure, df, df_product, 
                                               ch_list_graph, nutrients_choice, ["A", "B"])
            prevent_update = pnns2
            pnns1_chosen, pnns2_chosen = pnns1, pnns2
            graphic_gestion_style = {'display':'block'}
            
        except:
            selected_product_img = dash.get_asset_url("no_image.jpg")
            selected_product_title = html.Strong("Product not found, search for another one")
            selected_product_texte = html.Strong("The product is not available")

            title = dash.no_update
            subtitles = [dash.no_update] * len(diets)
            styles_images = [dash.no_update] * TOTAL_IMAGES
            textes_images = [dash.no_update] * TOTAL_IMAGES
            graphic_gestion_style = dash.no_update
            prevent_update = None
        
        selected_product_style = {'display':'block'}
        
    elif ctx.triggered_id == 'advanced_search_button':
        
        advanced_search_style = {'float': 'center', 'display': 'block', 'flex-direction': 'row', 
                                'width': '100%', 'background-color': '#F0F0F0', 'margin-bottom': '20px'}
        prevent_update = None
        
    # Top keep tract of the images src to load when clicking on 
    for i, src in enumerate(images):
        key = f'{diets[int(i/20)]}_img_{i%20}'
        if src == dash.no_update or src == None:
            continue
        else:
            shown_img_data[key] = src
            
    pnns1_chosen = pnns1_chosen if condition_selected_a_product else dash.no_update
    pnns2_chosen = pnns2_chosen if condition_selected_a_product else dash.no_update
            
    output_values = [title, *subtitles, *images, *styles_images, *textes_images, *others_img, *others_img_style,
                     figure, graphic_gestion_style, selected_diet, selected_product_style, 
                     selected_product_img, selected_product_title, selected_product_texte,
                     advanced_search_style, search_on, shown_img_data, history_nav, style_multiple_dropdown, 
                     browser_history_style, prevent_update, pnns1_chosen, pnns2_chosen, style_picture_search_div]
    
    print("display_images: ", time.time() - elapsed_time) if DEBUG else None  
    return tuple(output_values)

@app.callback(
    Output("selected_product_img", 'src', allow_duplicate=True),
    
    *[Input(f"selected_img_{i}", 'n_clicks') for i in range(4)],
    
    *[State(f"selected_img_{i}", 'src') for i in range(4)],
    
    prevent_initial_call=True,
)

def changing_image_selected_product(*args):
    images = args[4:]
    
    # Dictionary mapping trigger IDs to corresponding image indices
    trigger_to_index = {f"selected_img_{i}": i for i in range(4)}

    # Find the index of the triggered element
    triggered_index = trigger_to_index.get(ctx.triggered_id, None)

    if triggered_index is not None:
        return images[triggered_index]
    else:
        return dash.no_update

@app.callback(
    Output("graph_products_img", 'figure', allow_duplicate=True),
    
    Input('dropdown_nutrients_img', 'value'),
    Input('check_list_graph_img','value'),
    
    State('pnns1_chosen', 'data'),
    State('pnns2_chosen', 'data'),
    State('dropdown_country','value'),
    
    State('personnalized_sorting', 'data'),
    State('sliced_file', 'data'),
    State("dropdown_diet", "value"), 
    State('search_on', 'data'),
    State('dropdown_number_product', 'value'),
    
    prevent_initial_call=True,
)
# When modifying the graphic
def modifying_graph(nutrients_choice, ch_list_graph, pnns1_chosen, pnns2_chosen, country, 
                    selected_diet, df_slice, dropdown_diet, search_on, n_best):
    elapsed_time = time.time() if DEBUG else None
    if search_on:
        df = pd.read_json(StringIO(df_slice), orient='split')         
        df_N_best = df_sorting(dropdown_diet, df).head(n_best)
    else:
        df = return_df(country, pnns1_chosen, pnns2_chosen)
        df_N_best = df_sorting(selected_diet, df).head(n_best)
        
    if ctx.triggered_id == "dropdown_nutrients_img":
        # We check the nutrients_choice 
        nutrients_choice = nutrients_choice if nutrients_choice not in [None, []] else nutrients
        # We use the patch
        patched_figure = Patch()
        figure = patch_graphic(patched_figure, df, df_N_best, ch_list_graph, nutrients_choice, ["A", "B"])
    
    else : 
        figure = create_figure_products(df, nutrients, nutrients_choice, ch_list_graph, df_N_best)
    
    print("modifying_graph: ", time.time() - elapsed_time) if DEBUG else None 
    
    return figure

@app.callback(
    Output('arrow_button_panel', 'children'),
    Output('left_panel', 'style'),
    Output('left_panel_div2', 'style'),
    
    Input('arrow_button_panel', 'n_clicks'),
    
    prevent_initial_call=True,
)
def left_panel_display(n_clicks):
    # To display or not the left panel
    if n_clicks%2:
        style_panel={'background-color': '#F0F0F0', 'overflowY': 'scroll', 'height': '100vh', 'flex': '0.1', 'direction': 'ltr',
             'border-right': '1px solid black', 'margin':'0px'}
        style_panel_div={'display':'None'}
        #Unicode: U+2261 or html &#8801;
        return "≡", style_panel, style_panel_div 
    
    else:
        style_panel={'background-color': '#F0F0F0', 'overflowY': 'scroll', 'height': '100vh', 'flex': '1', 'direction': 'rtl',
             'border-right': '1px solid black', 'margin':'0px'}
        style_panel_div={'display':'block'}
        
        return "←", style_panel, style_panel_div
    
@app.callback(
    Output('images_gestion', 'style', allow_duplicate=True),
    Output('browser_history_div', 'style', allow_duplicate=True),
    Output('advanced_search_div', 'style', allow_duplicate=True),
    Output('browser_table', 'data'),
    Output('browser_table', 'columns'),
    Output('browser_table', 'selected_rows'),
    Output('dropdown_country','value', allow_duplicate=True),
    Output('pnns1_chosen','data', allow_duplicate=True),
    Output('pnns2_chosen','data', allow_duplicate=True),
    Output('loading_history','data'),
    Output('search_bar', 'value'),
    *[Output(f'{diet}_div', 'n_clicks') for diet in diets],
    
    Input('browsing_button','n_clicks'),
    Input('browser_table', 'selected_rows'),
    
    State('history', 'data'),
    State('dropdown_country', 'value'),
    State('type_search_product', 'value'),
    
    prevent_initial_call=True,
)

def browsing_history(clicks, selected_rows, history_nav, country, type_search_product):
    elapsed_time = time.time() if DEBUG else None
    
    pnns1, pnns2, loading_history, search_bar = dash.no_update, dash.no_update, dash.no_update, dash.no_update
    columns, df_history_nav = dash.no_update, dash.no_update
    clicks_diet = [dash.no_update] * len(diets)
    
    if ctx.triggered_id == "browsing_button":
        
        style_images_gestion = {'display':'None'}
        advanced_search_style = {'display':'None'}
        style_browser_history = {'display':'block'}
        selected_rows = []
        country = dash.no_update
        
        # transforming to Dataframe
        df_history_nav = pd.DataFrame(history_nav, columns = ["Type", "Country", "Pnns1", "Pnns2", "Diet", "Code"])

        columns=[{'name': col, 'id': col} for col in df_history_nav.columns]
        
        # I don't show the code
        df_history_nav=df_history_nav.to_dict('records')
        
    elif ctx.triggered_id == "browser_table":
        
        # We retrieve the row and it's data
        selected_history = history_nav[selected_rows[0]]
        
        # We modify the country, if the actual setup is not the same
        if selected_history[1] == country:
            country = dash.no_update
        else: 
            country = selected_history[1]
        
        if "Product:" in selected_history[0]:
            # We check the type of data currently selected
            if type_search_product == "Product name":
                # We eliminate "product: " to conserve the name of the product
                search_bar = selected_history[0].lstrip('Product: ')
            elif type_search_product == "Product code":
                search_bar = selected_history[5]
                
        elif "Navigation" in selected_history[0]:
            
            pnns1, pnns2, diet_choice = selected_history[2], selected_history[3], selected_history[4]
            
            # To signal a modification and help reload pnns button from left panel
            loading_history = True
            
            if diet_choice != None:
                
                # We modify the dash.component to trigger a diet like search
                diet_index = diets.index(diet_choice) if diet_choice in diets else None
                if diet_index is not None:
                    clicks_diet[diet_index] = 1
        
        style_images_gestion = {'display': 'block', 'flex-direction': 'row', 'width': '100%'}
        style_browser_history = {'display':'None'}            
            
    output_values = [style_images_gestion, style_browser_history, advanced_search_style,
                     df_history_nav, columns, selected_rows, country, pnns1, pnns2, 
                     loading_history, search_bar, *clicks_diet]
    
    print("browsing_history", time.time() - elapsed_time) if DEBUG else None  
    return tuple(output_values)

@app.callback(
    Output('picture_search_div', 'style', allow_duplicate=True),
    Output('advanced_search_div', 'style', allow_duplicate=True),
    Output('browser_history_div', 'style', allow_duplicate=True),
    Output('images_gestion', 'style', allow_duplicate=True),
    Output('upload_img_button','style'),
    Output('clear_img_button','style'),
    Output('search_pnns1_img','style'),
    Output('search_pnns2_img','style'),
    Output('result_model_image','style'),
    Output('uploaded_img','children'),
    Output('model_figure_result','figure'),
    Output('pnns1_chosen', 'data', allow_duplicate=True),
    Output('pnns2_chosen', 'data', allow_duplicate=True),
    
    Input('picture_search_button','n_clicks'),
    Input('upload_img_button','n_clicks'),
    Input('clear_img_button','n_clicks'),
    Input('search_pnns1_img','n_clicks'),
    Input('search_pnns2_img','n_clicks'),
    Input('upload_img_data', 'contents'),
    Input('model_figure_result','clickData'),
    
    prevent_initial_call=True,
)
# Will handle everything linked to the search by a picture
def picture_search_div(*args):
    elapsed_time = time.time() if DEBUG else None
    
    display_no_show = {'display':'None'}
    
    # We display the div
    style_div = {'float': 'center', 'display': 'block', 'flex-direction': 'row', 'width': '100%'}
    style_others_div = [display_no_show]*3
                     
    image_contents = args[-2]
    clicked_graphic = args[-1]
    
    uploaded_img_div = dash.no_update
    style_search_pnns1 = dash.no_update
    style_search_pnns2 = dash.no_update
    style_upload_button = dash.no_update
    style_clear_button = dash.no_update
    style_result_model = display_no_show
    figure = dash.no_update
    pnns1_chosen, pnns2_chosen = dash.no_update, dash.no_update 
    
    if ctx.triggered_id == "upload_img_button":
        uploaded_img_div = None
        image_contents = None
    
    elif ctx.triggered_id == "upload_img_data":
        style_upload_button = display_no_show
        style_clear_button = style16_nd
        style_search_pnns1 = style16_nd
        style_search_pnns2 = style16_nd
        
        uploaded_img_div = html.Img(src=image_contents, style={'width': '500px', 'height':'500px'})
        
    elif ctx.triggered_id in ["clear_img_button", "picture_search_button"]:
        style_upload_button = style16_nd
        style_clear_button = display_no_show
        style_search_pnns1 = display_no_show
        style_search_pnns2 = display_no_show
        uploaded_img_div = None
        image_contents = None
        
    elif ctx.triggered_id in ["search_pnns1_img", "search_pnns2_img"]:
        style_result_model = {'display': 'flex', 'flex-direction': 'column', 'width': '100%', 'margin-top':'20px',
                              'border-bottom': '1px solid black', 'border-top': '4px solid black'}
        image_contents = {"base64_image": image_contents}
        
        # We send the image to the model
        response = requests.post("http://localhost:8000/process-image/", data=image_contents)
        
        try:
            # We put the result into a dataframe
            df_result = pd.DataFrame(response.json())

            # We show the result with a graphic
            # For now, only 3 results, could be expanded with a button
            figure = figure_result_model(df_result.iloc[:3])
        
        except:
            html.A("Format not compatible")
        
    elif ctx.triggered_id == "model_figure_result":
        
        # We hide the div
        style_div = display_no_show
        
        # We extract the chosen result by the user 
        pnns_chosen = clicked_graphic["points"][0]["y"]
        
        # We attribute the pnns1 and 2 associated
        if pnns_chosen in pnns_groups_1:
            pnns1_chosen = pnns_chosen
            pnns2_chosen = None
        elif pnns_chosen in pnns_groups_2:
            pnns1_chosen = next((key for key, value in pnns_groups.items() if pnns_chosen in value), None)
            pnns2_chosen = pnns_chosen
    
    output_values = [style_div, *style_others_div, style_upload_button, style_clear_button, style_search_pnns1, 
                     style_search_pnns2, style_result_model, uploaded_img_div, figure, pnns1_chosen, pnns2_chosen]
    
    print("picture_search_div", time.time() - elapsed_time) if DEBUG else None  

    return tuple(output_values) 

# Run the app
if __name__ == '__main__':
    app.run(debug=True)
    

click_pnns_showing 8.106231689453125e-05
search_bar_option_def 0.4396660327911377
display_images:  1.7510137557983398
click_pnns_showing 0.0008602142333984375
search_bar_option_def 0.46346402168273926
picture_search_div 4.100799560546875e-05
display_images:  1.3541619777679443
picture_search_div 2.4080276489257812e-05
picture_search_div 8.869171142578125e-05
picture_search_div 0.1780998706817627
picture_search_div 5.3882598876953125e-05
click_pnns_showing 9.393692016601562e-05
display_images:  0.5660159587860107


In [85]:
def patch_graphic(patched_figure, df, df_product, ch_list_graph, type_modification):
    """
        Function that modify the patch figure
        Take as arguments, the dataframes to use
        The type of figure
        The type of modification (list):
        A: Change for primary data
        B: Change to the product data
        
        Return the patched_figure
    """
    if "A" in type_modification:
        
    elif "B" in type_modification:
    
    re
                
                if ctx.triggered_id == "selected_product_table":
                    df = pd.read_json(StringIO(df_selected_product), orient='split')
                
                else:
                    df = df_slice
                    # We change the title when too much data is changing
                    patched_figure['layout']['title']['text'] = f'Distribution of macronutrients of selected products [{df_slice.shape[0]}]'
            
                if nutrients_choice not in [None, []]:
                    nutrients_list = nutrients_choice
                else:
                    nutrients_list = nutrients
                
                product_name_list = [[value] for value in df["product_name"].values]
                # Changing the title 
                
                if ch_list_graph in ["Distribution", "Products"]:
                    if ctx.triggered_id !=  "selected_product_table":
                        A, B = 0, 1
                    else:
                        A, B = 2, 3
                        
                    patched_figure['data'][A]['customdata'] = product_name_list
                    patched_figure['data'][B]['customdata'] = product_name_list * len(nutrients_list)

                    patched_figure['data'][A]['y'] = [value for value in df["energy_100g"].values]
                    patched_figure['data'][B]['x'] = [nut for nut in nutrients_list for value in df[nut].values]
                    patched_figure['data'][B]['y'] = [value for nut in nutrients_list for value in df[nut].values]

                elif ch_list_graph == "Radarplot" and ctx.triggered_id ==  "selected_product_table":
                    
                    patched_figure = create_figure_products(df_slice, nutrients_list, nutrients_list, ch_list_graph, df)
                    
                else: 
                    # For modifiying the median values 
                    # theta: nutrients names
                    # r: values
                    # -1: median position in the figure
                    median_df = df[nutrients_list].median()

                    # We repeat the first at the end to close the radarplot
                    values = median_df.values.tolist() + [median_df.values[0]]
                    columns = median_df.index.tolist() + [median_df.index[0]]

                    patched_figure['data'][-1]['theta'] = columns
                    patched_figure['data'][-1]['r'] = values

                    patched_figure['data'][-1]['name']= f'Median of {df_slice.shape[0]} products'
                
                print("graph_macronutrients 2", time.time() - elapsed_time) if DEBUG else None
                return patched_figure, initialization_graph
            elif ctx.triggered_id == "dropdown_nutrients":
                
                # We take a Patch() to modify only some elements of the figure
                patched_figure = Patch()
                if df_selected_product != None :
                    dffs = [df_slice, pd.read_json(StringIO(df_selected_product), orient='split')]
                else :
                    dffs = [df_slice]
                for i, df in enumerate(dffs):
                
                    if nutrients_choice not in [None, []]:
                        nutrients_list = nutrients_choice
                    else:
                        nutrients_list = nutrients
                
                    product_name_list = [[value] for value in df["product_name"].values]
                    # Changing the title 

                    if ch_list_graph in ["Distribution", "Products"]:
                        if i == 0:
                            A, B = 0, 1
                        else:
                            A, B = 2, 3
                            
                        patched_figure['data'][A]['customdata'] = product_name_list
                        patched_figure['data'][B]['customdata'] = product_name_list * len(nutrients_list)

                        patched_figure['data'][A]['y'] = [value for value in df["energy_100g"].values]
                        patched_figure['data'][B]['x'] = [nut for nut in nutrients_list for value in df[nut].values]
                        patched_figure['data'][B]['y'] = [value for nut in nutrients_list for value in df[nut].values]

                    elif ch_list_graph == "Radarplot" and i == 1:
                        if df.shape[0] > 0 :
                            patched_figure = create_figure_products(df_slice, nutrients_list, nutrients_list, ch_list_graph, df)
                            return patched_figure, initialization_graph
                    else: 
                        # For modifiying the median values 
                        # theta: nutrients names
                        # r: values
                        # -1: median position in the figure
                        median_df = df[nutrients_list].median()

                        # We repeat the first at the end to close the radarplot
                        values = median_df.values.tolist() + [median_df.values[0]]
                        columns = median_df.index.tolist() + [median_df.index[0]]

                        patched_figure['data'][-1]['theta'] = columns
                        patched_figure['data'][-1]['r'] = values

                        patched_figure['data'][-1]['name']= f'Median of {df_slice.shape[0]} products'

                print("graph_macronutrients 2", time.time() - elapsed_time) if DEBUG else None
                return patched_figure, initialization_graph
            

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 33)